In [1]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [2]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[0, 1, 2], data_path='.', round_num=3):
    profits = []
    for day in days:

        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [3]:
def backtest_days_r3(trader_merle, param_grid, days=[0, 1, 2], data_path='.', round_num=3):
    profits = []
    for day in days:
        # add "day": day to param_grid
        param_grid['day'] = day
        trader = trader_merle.Trader(param_grid)

        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits


In [5]:
import trader_merle

reload(trader_merle)

class Product:
    RESIN  = "RAINFOREST_RESIN"
    KELP   = "KELP"
    INK    = "SQUID_INK"

    CROISSANTS = "CROISSANTS"
    JAMS       = "JAMS"
    DJEMBES    = "DJEMBES"

    PB1 = "PICNIC_BASKET1"
    PB2 = "PICNIC_BASKET2"

param_grid = {
    # "9500_threshold": [0.25, 0.5, 1, 1.5, 2, 2.5, 3, 3.5,4 ,5],
    # "9500_max_qty": [5, 10, 20, 50, 100, 200],
    # "9750_threshold": [0.25, 0.5, 1, 1.5, 2, 2.5, 3, 3.5,4 ,5],
    # "9750_max_qty": [5, 10, 20, 50, 100, 200],
    "10500_threshold": [0.25, 0.5, 1, 1.5, 2, 2.5, 3, 3.5,4 ,5],
    "10500_max_qty": [5, 10, 20, 50, 100, 200],
}


param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    days = [0,1,2]
    profits = backtest_days_r3(trader_merle, params, days=days, data_path='.', round_num=3)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 60/60 [07:52<00:00,  7.87s/it]


In [6]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,10500_threshold,10500_max_qty,day,profit_day_0,profit_day_1,profit_day_2,tot_profit
0,0.25,5,2,-23696.0,58315.5,-6917.0,27702.5
1,0.25,10,2,-23696.0,58315.5,-6917.0,27702.5
32,2.50,20,2,-23696.0,58315.5,-6917.0,27702.5
33,2.50,50,2,-23696.0,58315.5,-6917.0,27702.5
34,2.50,100,2,-23696.0,58315.5,-6917.0,27702.5
35,2.50,200,2,-23696.0,58315.5,-6917.0,27702.5
36,3.00,5,2,-23696.0,58315.5,-6917.0,27702.5
37,3.00,10,2,-23696.0,58315.5,-6917.0,27702.5
38,3.00,20,2,-23696.0,58315.5,-6917.0,27702.5
39,3.00,50,2,-23696.0,58315.5,-6917.0,27702.5


In [8]:
df

,10500_threshold,10500_max_qty,profit_day_2,tot_profit
20,2.50,5,0.0,0.0
30,3.50,20,0.0,0.0
23,2.50,50,0.0,0.0
24,3.00,5,0.0,0.0
25,3.00,10,0.0,0.0
26,3.00,20,0.0,0.0
27,3.00,50,0.0,0.0
28,3.50,5,0.0,0.0
29,3.50,10,0.0,0.0
31,3.50,50,0.0,0.0


In [16]:
# print all column with all 3 days being profitable
profitable_days = df[(df['profit_day_-2'] > 0) & (df['profit_day_-1'] > 0) & (df['profit_day_0'] > 0)]

profitable_days

,ink_change_threshold_pct,ink_window_size,ink_position_limit,ignore_window_size,price_percent_needed,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
417,0.030,500,50,50,0.03,1357.0,5283.0,1755.0,8395.0
418,0.030,500,50,50,0.05,1357.0,5283.0,1755.0,8395.0
263,0.020,25,50,20,0.05,2295.0,5656.0,287.5,8238.5
262,0.020,25,50,20,0.03,2295.0,5656.0,287.5,8238.5
259,0.020,25,50,10,0.07,2295.0,5520.0,418.5,8233.5
...,...,...,...,...,...,...,...,...,...
152,0.015,25,50,10,0.03,14.0,126.0,165.0,305.0
153,0.015,25,50,10,0.05,14.0,126.0,165.0,305.0
150,0.015,25,50,10,0.10,14.0,14.0,165.0,193.0
151,0.015,25,50,10,0.20,14.0,14.0,165.0,193.0
